# Offensive Line EDA
### Sol Ben-Ishay

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
# Read in data
games_df = pd.read_csv("../Data/games.csv")
plays_df = pd.read_csv("../Data/plays.csv")
players_df = pd.read_csv("../Data/players.csv")
scouting_df = pd.read_csv("../Data/pffScoutingData.csv")
week1_tracking_df = pd.read_csv("../Data/week1.csv")
week2_tracking_df = pd.read_csv("../Data/week2.csv")

In [ ]:
#Get main tracking data
coi = ['gameId', 'season', 'week', 'playId',
       'playDescription',  'frameId', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'defensiveTeam', 'passResult', 'prePenaltyPlayResult',
       'absoluteYardlineNumber', 'dropBackType', 'pff_playAction', 'nflId', 'time', 'team',
       'x', 'y', 'event', 'officialPosition', 'displayName']

gptp_df = (games_df.merge(plays_df, on='gameId', how='left')
                .merge(week1_tracking_df, on=['gameId','playId'], how='left')
                .merge(players_df, on= 'nflId', how='left'))[coi]

# Convert up dt_time column to time
gptp_df['time'] = pd.to_datetime(gptp_df['time'])

# Drop where frameId is NA (not sure what that means, assume it is where the tracking didn't join with the player data)
gptp_df = gptp_df.dropna(axis=0, how='all', subset=['frameId', 'time', 'team', 'x', 'y', 'event'])

# Replace officialPosition, diplayName, and nflId with ball/0
gptp_df = gptp_df.fillna(value={'nflId': 0, 'officialPosition': 'ball', 'displayName': 'ball', 'dropBackType': 'none'})

In [ ]:
# Check for na data
if sum(gptp_df.isna().sum().values) != 0:
    print(gptp_df.isna().sum())

In [ ]:
gptp_df['passResult'].unique()
# Potential snap events: ['ball_snap', 'autoevent_ballsnap']
# Potential action events: ['pass_forward', 'autoevent_passforward', 'autoevent_passinterrupted', 'qb_sack', 'fumble', 'run', 'qb_strip_sack', 'lateral', 'handoff']
# Potential drop back types: ['TRADITIONAL', 'SCRAMBLE_ROLLOUT_RIGHT', 'DESIGNED_ROLLOUT_RIGHT', 'nan', 'SCRAMBLE', 'DESIGNED_ROLLOUT_LEFT', 'UNKNOWN', 'DESIGNED_RUN', 'SCRAMBLE_ROLLOUT_LEFT']

In [ ]:
# Get time from snap to action event (using ball)
potential_snap_events = ['ball_snap', 'autoevent_ballsnap']
potential_action_events = ['pass_forward', 'autoevent_passforward', 'autoevent_passinterrupted', 'qb_sack', 'fumble', 'run', 'qb_strip_sack', 'lateral', 'handoff']

coi = ['gameId', 'playId', 'time']
snap_to_action_df = gptp_df.query(f"(nflId == 0) and (event in {potential_snap_events} or event in {potential_action_events})")[coi]
snap_to_action_df = snap_to_action_df.groupby(['gameId', 'playId']).apply(lambda x: max(x['time']) - min(x['time'])).reset_index(name = 'snap_to_action')
snap_to_action_df['snap_to_action'] = snap_to_action_df['snap_to_action'].dt.total_seconds()
snap_to_action_df.head()

coi = ['gameId', 'season', 'week', 'playId', 'quarter', 'down', 'possessionTeam', 'team', 'yardsToGo', 'passResult', 'prePenaltyPlayResult']
play_result_df = gptp_df.query("officialPosition == 'ball' and (frameId == 1)")[coi]
play_result_df

snap_to_action_df = play_result_df.merge(snap_to_action_df, on=['gameId', 'playId'], how = 'left').drop([68, 169, 587, 311])
# Drop bad rows (weird snap_to_action times for variety of reasons)
# snap_to_action_df.sort_values(by=['snap_to_action'], ascending=[True]).head()
snap_to_action_df.head()

In [ ]:
import seaborn as sns
box = sns.boxplot(data=snap_to_action_df, x="passResult", y="snap_to_action")
display(box)

In [ ]:
strip = sns.stripplot(data=snap_to_action_df, x="passResult", y="snap_to_action")
display(strip)